**Mount Google Drive & Setup**

In [1]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

# Define paths
REPO_PATH = "/content/PlanckDeep"
DATA_PATH = "/content/PlanckDeep/data"
DRIVE_DATA_DIR = "/content/drive/MyDrive/PlanckDeep/processed_data"

# Clone your repo (if not done already)
!git clone https://github.com/KhamessiTaha/PlanckDeep.git

# Create /data directory inside the repo
os.makedirs(DATA_PATH, exist_ok=True)

# Copy processed .npy files from Drive to /data
!cp {DRIVE_DATA_DIR}/*.npy {DATA_PATH}/


Mounted at /content/drive
Cloning into 'PlanckDeep'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 71 (delta 33), reused 53 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (71/71), 3.08 MiB | 27.66 MiB/s, done.
Resolving deltas: 100% (33/33), done.


**Install dependencies**

In [2]:
!pip install numpy matplotlib scikit-learn torch torchvision tqdm seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

**Check data loaded properly**

In [3]:
import numpy as np
import os

# Check loaded files
patches = np.load(os.path.join(DATA_PATH, "cmb_patches_enhanced.npy"))
labels = np.load(os.path.join(DATA_PATH, "cmb_temperature_labels.npy"))

print("✓ Loaded patches:", patches.shape)
print("✓ Loaded labels:", labels.shape)


✓ Loaded patches: (5000, 128, 128)
✓ Loaded labels: (5000,)


**Train models from script**

In [4]:
%cd /content/PlanckDeep/src

!python train_all_models.py \
  --patch_file ../data/cmb_patches_enhanced.npy \
  --label_file ../data/cmb_temperature_labels.npy \
  --models improved_cnn resnet physics_informed \
  --epochs 20 \
  --batch_size 32 \
  --use_class_weights \
  --use_early_stopping \
  --lr_scheduler cosine \
  --experiment_name deep_cmb_colab_run


/content/PlanckDeep/src
Using device: cpu
Experiment directory: experiments/deep_cmb_colab_run
Loading and preparing datasets...
Loaded 5000 patches with shape (128, 128)
Number of classes: 2
Class distribution: [3332 1668]
Dataset loaded: 5000 patches
Patch shape: torch.Size([1, 128, 128])
Label distribution: tensor([3332, 1668])
Dataset splits - Train: 3500, Val: 1000, Test: 500
Class weights: tensor([0.7475, 1.5099])

Training IMPROVED_CNN Model

Creating improved_cnn model...
Model parameters: 10,649,538 total, 10,649,538 trainable
Epoch 1/20 (327.72s) - Train Loss: 0.696917, Train Acc: 0.5203, Val Loss: 0.693207, Val Acc: 0.3281, LR: 1.00e-03
Traceback (most recent call last):
  File "/content/PlanckDeep/src/train_all_models.py", line 553, in <module>
    main()
  File "/content/PlanckDeep/src/train_all_models.py", line 518, in main
    model, results, history = train_single_model(
                              ^^^^^^^^^^^^^^^^^^^
  File "/content/PlanckDeep/src/train_all_models.p

**Save experiment results back to Drive**

In [ ]:
# Copy the full experiment directory back to Drive
EXPERIMENT_NAME = "deep_cmb_colab_run"
SRC = f"experiments/{EXPERIMENT_NAME}"
DST = f"/content/drive/MyDrive/PlanckDeep/experiments/{EXPERIMENT_NAME}"

!mkdir -p "{DST}"
!cp -r {SRC}/* "{DST}/"

print("✓ Results copied to Drive:")
print(DST)


In [9]:
!git pull https://github.com/KhamessiTaha/PlanckDeep.git

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 657 bytes | 328.00 KiB/s, done.
From https://github.com/KhamessiTaha/PlanckDeep
 * branch            HEAD       -> FETCH_HEAD
Updating c38be90..cd129e2
Fast-forward
 src/utils.py | 23 ++++++++++++++++-------
 1 file changed, 16 insertions(+), 7 deletions(-)
